In [ ]:
#通过Bio.SeqIO读取数据
from Bio.SeqIO.QualityIO import FastqGeneralIterator
seq_t0 = []
seq_t3 = []
for title, seq, qual in FastqGeneralIterator('C:/Users/lch/Desktop/python期末作业/project3/project3/t0.fq'):
    seq_t0.append(seq)
for title, seq, qual in FastqGeneralIterator('C:/Users/lch/Desktop/python期末作业/project3/project3/t3.fq'):
    seq_t3.append(seq)

import pandas as pd
import numpy as np
import csv

#读取barcode_reference为b_ref
b_ref = pd.read_csv("C:/Users/lch/Desktop/python期末作业/project3/project3/barcode_reference.csv",header=0)
b_ref_stat = pd.read_csv("C:/Users/lch/Desktop/python期末作业/project3/project3/barcode_reference.csv",index_col=0,header=0)

#建一个DataFrame，统计包括ref_barcode、t0出现次数（初始为0）、t3出现次数（初始为0）

arr_need = np.zeros((len(b_ref.iloc[:,0]),2),dtype=int)
ori_DF = pd.DataFrame(arr_need,index=b_ref.iloc[:,0],columns=["t0","t3"])
bref_2 = pd.concat([b_ref_stat,ori_DF],axis = 1)
bref_2 = bref_2[bref_2["position"]!="wt"]

for i in bref_2.index:
    bref_2["t0"].loc[i]+=seq_t0.count(i)
    bref_2["t3"].loc[i]+=seq_t3.count(i)
    print(bref_2.loc[i])

bref_2.to_csv("C:/Users/lch/Desktop/bref_2.csv")

from scipy import stats
import math
#读取barcode_reference为b_ref，之前统计好的barcode出现次数数据为bref_2 
b_ref = pd.read_csv("C:/Users/lch/Desktop/python期末作业/project3/project3/barcode_reference.csv",header=0)
bref_2 = pd.read_csv("C:/Users/lch/Desktop/bref_2.csv",index_col=0,header=0)

#读codon表为codon_dict
with open("C:/Users/lch/Desktop/python期末作业/project3/project3/codon_table.csv", "r", encoding='utf-8') as codon_table:
    list_csv = list(csv.reader(codon_table))
    codon_dict = {}
        # 利用for循环将表中氨基酸对应序列、氨基酸名称缩写、氨基酸名称写入字典
    for i in range(0, len(list_csv)):
        codon_dict[list_csv[i][0]] = [list_csv[i][1], list_csv[i][2]]
#读取野生型序列，三个三个排为WTseq
with open("C:/Users/lch/Desktop/python期末作业/project3/project3/WT_seq.txt","r") as WT_seq:
    WTseq_0=list(WT_seq.read())
    WTseq = []
    for i in range(1,int(len(WTseq_0)/3+1)):
        num_int = int(i*3-3)
        codon_x = "".join(WTseq_0[num_int]+WTseq_0[num_int+1]+WTseq_0[num_int+2])
        WTseq.append(codon_x)
#通过codon表将密码子转化为氨基酸，转化后的突变氨基酸作为新列输入DataFrame
arr_need = np.full((len(bref_2.index),1),np.nan)
pvalue_x = pd.DataFrame(arr_need,index=bref_2.index,columns=["amino_mut"])
bref_2 = pd.concat([bref_2,pvalue_x],axis = 1)
for i in bref_2.index:
    single_bref = bref_2.loc[i]
    bref_2["amino_mut"].loc[i]=codon_dict[single_bref["mutant"]][0]
#将相同position、相同突变氨基酸的行通过groupby聚合在一起
iuc = bref_2.groupby([bref_2["position"],bref_2["amino_mut"]]).sum()#agg({"t0":"sum","t3":"sum","pvalue":"sum"})
#建立一个空矩阵填充入DataFrame，合并入"amino_wt","pvalue","t3_vs_t0"三列
arr_need = np.full((len(iuc.index),3),np.nan)
pvalue_x = pd.DataFrame(arr_need,index=iuc.index,columns=["amino_wt","pvalue","t3_vs_t0"])
iuc = pd.concat([iuc,pvalue_x],axis = 1)
#获取数据均一化所需的t0、t3 reads总数
t0_sum=len(seq_t0)
t3_sum=len(seq_t3)
print("t0:",t0_sum,"t3:",t3_sum)
#填充数据入空白的三列，分别是：通过合并前数据求取的pvalue值、wt密码子对应的氨基酸、t3/t0氨基酸出现频率的比较（公式：log2（t3/t3_sum/t0*t0_sum+1e-04））
for i,z in iuc.index.tolist():
    single_bref =bref_2[bref_2["position"]==i][bref_2["amino_mut"]==z]
    iuc["pvalue"].loc[i].loc[z]=stats.ttest_ind(a=single_bref["t0"],b=single_bref["t3"],equal_var=True)[1]
    iuc["amino_wt"].loc[i].loc[z]=codon_dict[single_bref["original"][0]][0]
    iuc["t3_vs_t0"].loc[i].loc[z]=math.log(iuc["t3"].loc[i].loc[z]/iuc["t0"].loc[i].loc[z]/t3_sum*t0_sum+1e-04,2)
#输出到新的csv
iuc.to_csv("C:/Users/lch/Desktop/iuc.csv")

iuc = pd.read_csv("C:/Users/lch/Desktop/iuc.csv",header=0)
#重新写入之前未写入的wt的氨基酸，输出为新的csv
for i in iuc.index:
    iuc["amino_wt"].loc[i]=codon_dict[WTseq[iuc["position"].loc[i]-1]][0]
    print(WTseq[iuc["position"].loc[i]-1])
iuc.set_index("position",drop=True)
print(iuc)
iuc.to_csv("C:/Users/lch/Desktop/iuc2.csv")
